# Scraping Mars Web Data

In [2]:
# Dependencies
import pandas as pd
import requests
from splinter import Browser
from bs4 import BeautifulSoup

### NASA Mars News Site
* latest News Title
* paragraph text
* source: https://mars.nasa.gov/news/

In [ ]:
# variable names
# news_title
# news_p

In [33]:
# Set up path and url
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
news_url = 'https://mars.nasa.gov/news/'

# Visit url and retrieve first article title and description
browser.visit(news_url)
news_html = browser.html
soup = BeautifulSoup(news_html, 'html.parser')
news_title = soup.find('div', class_='bottom_gradient')
news_title_2 = news_title.find('h3').text
news_p = soup.find('div', class_='article_teaser_body').text
print(news_title_2)
print(news_p)
browser.quit()

Hear Audio From NASA's Perseverance As It Travels Through Deep Space
The first to be rigged with microphones, the agency's latest Mars rover picked up the subtle sounds of its own inner workings during interplanetary flight.


### JPL Mars Space Images
* URL for featured Mars image (full size jpg image)
* Source: https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars

In [ ]:
# variable name: featured_image_url

### Mars Facts
* table containing facts about the planet
* Source: https://space-facts.com/mars/

In [ ]:
# Use Pandas to convert to HTML table string

### Mars Hemispheres
* High resolution images of each of Mar's hemispheres
* Source: https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars

In [ ]:
# Will need to click each of the links to the hemispheres (for full resolution)
# variable names
# list of dictionaries with two keys in each dict: 
# img_url
# title